# Geocoding: Determining the Geographic Coordinates of Street Addresses

Geocoding is the process of determining the geographic coordinates of a place name, code, or street address. In this exercise we focus on street addresses. This notebook explores this process using the **geopy** library. 

Address geocoding requires a set of *properly formatted addresses* as input. The proper format for these addresses is the one required by the geocoding software.  The geocoding software parses the input addresses and compares them to a reference database of street, parcel, or building locations. Reference databases are typically street centerlines that contain the start and end addresses for each side of the street for each street block. Once the street block that matches the input address is found, linear interpolation is used to estimate the coordinates.

Needless to say, some geocoders do better than others at estimating location. The accuracy is highly dependent on the accuracy of the underlying reference database as well as the sophistication of a geocoder's algorithms for parsing addresses and interpolating location.  Let's take a look at this process and map the results.

In [ ]:
# First, load libraries
from datascience import *
from geopy import *
from geopy.distance import vincenty
import numpy as np
import folium

# Simple function to show folium maps inline
from IPython.display import HTML

def inline_map(m, height=500):
    """Takes a folium instance and embed HTML."""
    m._build_map()
    srcdoc = m.HTML.replace('"', '&quot;')
    embed = HTML('<iframe srcdoc="{0}" '
                 'style="width: 100%; height: {1}px; '
                 'border: none"></iframe>'.format(srcdoc, height))
    return embed

## The Addresses to Geocode
First, read in the file of ten addresses. Take a minute to look at the CSV file to see how the addresses are formatted.

In [ ]:
my_addresses = Table.read_table('ten_addresses_comma.csv')
my_addresses


### Nominatim - the OpenStreetMap Geocoder

Take a look at the [Geopy](https://geopy.readthedocs.org/en/1.10.0/) documentation for geocoding using the [Nominatim](http://wiki.openstreetmap.org/wiki/Nominatim) geocoder. Nominatim uses OpenStreetMap data as the reference database. We like Nominatim because it is based on free and open source (FOSS) data and does not require an API key. Moreover, if you use one of Nominatim's partner server [MapquestOpen](http://open.mapquestapi.com/geocoding/) there are no restrictions on the number of addresses you can geocode or the ways in which you can use the geocoding results.

We will start by geocoding one address using the Geopy library to access Nominatim.

In [ ]:
# Fetch one address from our table
one_address = my_addresses['address'][0]
print("Here is the input address:", one_address)

nominatim_geolocator = Nominatim() # create a Nominatim geocoder object

nlocation = nominatim_geolocator.geocode(one_address) # geocode our one address
print("\nHere is the address that Nominatim geocoded:\n", nlocation.address)
print("\nAddress latitude: %.4f and longitude: %.4f" % (nlocation.latitude, nlocation.longitude))
print("\nRaw Nominatim ouput:\n",nlocation.raw)

### Interpreting the output
You can see from above that the geocoder standardizes the address before geocoding. It also returns a lot of output for each address. You can read about each value returned on the Nominatim geocoding page.

## Reverse Geocoding
Reverse geocoding is the process of determining the address for geographic coordinates. You can also do this with the Nominatim geocoder. See the example below.

In [ ]:
nominatim_geolocator.reverse((37.7548032228341, -122.124999975319))
 

## Batch Geocoding

Now that we have figured out how to use the Nominatim geocoder, let's create a function so that we can apply it to a table, or batch, of addresses and then map the output.  This process is shown below.

In [ ]:
# A simple function to get the coordinates for one address
def getPoint(one_address, the_locator):
    print('Using [%s] to geocode this address [%s]'% (the_locator.api,one_address))
    location = the_locator.geocode(one_address)
    pointAsArray =  (location.latitude, location.longitude)
    return pointAsArray

In [ ]:
#Test the function with our one address
getPoint(one_address, nominatim_geolocator)

## Storing our results
Let's apply our function to the table of addresses and store the output in a new column in the table called **nom_geom** where **nom** reminds us that the coordinates are from the Nominatim geocoding service.  Since geocoding is a service that uses system resources and which typically is limited, one should always save the results when that is allowed by the terms of service, rather than repeatedly requesting the same data.

In [ ]:
# Apply the function to our table of addresses
my_addresses['nom_geom'] = my_addresses.apply(lambda x: getPoint(x,nominatim_geolocator), ['address'])

In [ ]:
my_addresses

### Map the results
We can write a  little function to map the results of the geocoded addresses. By adding a popup we can see what the address is at each location.

In [ ]:
def mapMyPoint(the_map, the_point, the_address, m_color='blue'):
    the_map.simple_marker(location=the_point, popup=the_address, marker_color=m_color)

In [ ]:
ctr_lat = np.mean(my_addresses['nom_geom'][:, 0]) # find the average of all latitude values in the array
ctr_lon = np.mean(my_addresses['nom_geom'][:, 1]) # find the average of all longitude values in the array

m = folium.Map([ctr_lat, ctr_lon], zoom_start=15)

my_addresses.apply(lambda x,y: mapMyPoint(m, x,y), ['nom_geom','address'])
 
inline_map(m)

## Saving your output
Once you have geocoded a set of addresses you typically want to save your output. The Tables module has a **to_csv** method but it cannot write columns that contain arrays like *nom_geom*. So we will create a lat and lon column to hold those values for each address.

In [ ]:
my_addresses['nom_lon'] = my_addresses['nom_geom'][:,1]
my_addresses['nom_lat'] = my_addresses['nom_geom'][:,0]
my_addresses

In [ ]:
# Subset the table before writing
my_address_coords = my_addresses.select(['id','address','nom_lon','nom_lat'])
my_address_coords.to_csv('my_geocoded_addresses.csv')

## The Google Geocoder

The [Google Geocoder](https://developers.google.com/maps/documentation/geocoding/) is the gold standard for hiqh quality geocoding.  This service does not come cheap! Google limits folks to 2,500 addresses per day for free geocoding. That can be a significant barrier for many business and research applications where 1 million or more addresses may need geocoding. Before paying for geocoding services, it is a good idea to see how the results differ from free services like Nominatim.

The code below geocodes one address using Google Geocoder and then uses the Geopy's Vincenty function to compute the distance between the location returned by Google and by Nominatim.

In [ ]:
# Geocode one address with the google geocoder
print("Here is the input address:", one_address)

google_geolocator = GoogleV3() # create a Google geocoder object

glocation = google_geolocator.geocode(one_address) # geocode our one address
print("\nHere is the address that Google geocoded:\n", glocation.address)
print("\nAddress latitude: %.4f and longitude: %.4f" % (glocation.latitude, glocation.longitude))
print("\nRaw Google ouput:\n",glocation.raw)

### Google Geocoder Output
Take a look at the output returned by the Google geocoder above. It is much more extensive than that from Nominatim. Take a look at the Google docs for details.

## Question 1
What is the value for **glocation.raw['geometry']['location_type']** and what does it mean?


### Double-click on this markdown cell to input your answer to Question 1.



## Computing Vincenty Distance
We used the geopy's vincenty function in an earlier exercise to compute ellipsoidal distances. Let's use it now to compare geocoder results.

In [ ]:
# Compute the distance between the Google and Nominatim Output
gloc = (glocation.latitude, glocation.longitude)
nloc = (nlocation.latitude, nlocation.longitude)
print(vincenty(nloc, gloc).feet)

### Mapping the difference
Wow, a 120 feet difference in the geographic locations returned by the two geocoders can be quite signifcant depending on your application.  Let's map the difference between these two points for the same address.


In [ ]:
# Add the google geocoded address location to the map we previously created.
m.simple_marker(location=gloc, popup=glocation.address, marker_color='red')
inline_map(m)

## Question 2
- Geocode the my_addresses table using the Google Geolocator. 
- Add the output coordinates to the my_addresses table.
- Compute the vincenty difference in feet between the Google and Nominatim results for all addresses in the my_address table and add those differences to the my_addresses table.
- Map the Google and Nominatim locations on the same map using two different marker colors.

Show your work in the cells below. Use as many cells as you need - add more if needed.

## Question 3
- 3.a. What is the average difference in feet for the address locations returned by the two geocoders?
- 3.b. What address has the greatest difference in location returned by the two geocoders?
- 3.c. Can you think of any reasons why this address may be difficult to geocode?

In [ ]:
# Input your code for 3.a and 3.b in this cell


### Double-click on this markdown cell to input your answer to Question 3.c.


## Question 4

- Create a CSV file of addresses of the 19 Oakland Public Libraries listed on this website: http://oaklandlibrary.org/using-library/locations-hours.  The library addresses are also listed on this [PDF](http://www.oaklandlibrary.org/sites/default/files/uploads/location%20guide%2002_21_13%20inside_0.pdf) file.
 - You can create the CSV file by creating a new Google Sheet document and saving as CSV.
- Read the CSV file into this notebook.
- Geocode your addresses using the Google geocoder.
- Create a map of your geocoded addresses.
- Output the geocoded addresses to a CSV file with the latitude and longitude values returned by the geocoder.
- Show your work in the cells below. Add more cells if needed


## What to submit via bCourses

Submit a zip file containing the following:
- your completed notebook as an *.ipynb document
- your notebook as a PDF file
- your output CSV file from Question 4

### Due Date: Monday, March 14, 2016 at 4pm.
